In [3]:
(define (square n) (* n n))
(define (divides? a b) (= (remainder b a) 0))
(define (inc n) (+ 1 n))
(define (smallest-divisor n)
  (define (find-divisor test-divisor)
    (cond ((> (square test-divisor) n) n)
          ((divides? test-divisor n) test-divisor)
          (else (find-divisor (inc test-divisor)))))
  (find-divisor 2))

(define (prime? n) (= n (smallest-divisor n)))
(prime? 12)

#f

### Exercise 1.21: 
Use the smallest-divisor procedure to find the smallest divisor of each of the following numbers: `199, 1999, 19999`
#### Answer:

In [4]:
(smallest-divisor 199)

199

In [5]:
(smallest-divisor 1999)

1999

In [6]:
(smallest-divisor 19999)

7

### Exercise 1.22: 
Most Lisp implementations include a primitive called `runtime` that returns an integer that specifies the amount of time the system has been running (measured, for example, in microseconds). The following `timed-prime-test` procedure, when called with an integer *n*, prints *n* and checks to see if *n* is prime. If *n* is prime, the procedure prints three asterisks followed by the amount of time used in performing the test.
```scheme
(define (timed-prime-test n)
  (newline)
  (display n)
  (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
  (if (prime? n)
      (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
  (display " *** ")
  (display elapsed-time))
```
Using this procedure, write a procedure `search-for-primes` that checks the primality of consecutive odd integers in a specified range. Use your procedure to find the three smallest primes larger than 1000; larger than 10,000; larger than 100,000; larger than 1,000,000. Note the time needed to test each prime. Since the testing algorithm has order of growth of $\theta (\sqrt{n})$, you should expect that testing for primes around 10,000 should take about $\sqrt{10}$ times as long as testing for primes around 1000. Do your timing data bear this out? How well do the data for 100,000 and 1,000,000 support the $\theta (\sqrt{n})$ prediction? Is your result compatible with the notion that programs on your machine run in time proportional to the number of steps required for the computation?

#### Answer:
I've made some tweaks to the `timed-prime-test` function, such that
1. It prints numbers *only* when they are prime
2. It returns `true` if the number is prime, `false` otherwise.

the procedure is below:

In [7]:
; iracket (the racket kernel used in this notebook) does not support #lang changes.
; That is, #lang is a syntax for whole modules, whereas both the REPL and Jupyter
; work with top-level forms and generally receive them one at a time.
; Hence this workaround. Found this in sicp-solutions.net.
(define (runtime) (current-inexact-milliseconds))

(define (timed-prime-test n)
  (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
  (if (prime? n)
      (report-prime n (- (runtime) start-time))
      #f))
(define (report-prime n elapsed-time)
  (display n)
  (display " *** ")
  (display elapsed-time)
  (newline)
  #t)

Given below is a procedure that checks primality of odd integers within a range - `start` and `end`. It also takes a `limit` parameter which indicates the number of prime numbers the procedure should print and exit. The procedure will stop when we reach the `limit` or when we reach the end of the range.

In [8]:
(define (search-for-primes limit start end)
  (cond ((= limit 0) #t)
        ((> start end) #f)
        ((timed-prime-test start) (search-for-primes (- limit 1) (+ 2 start) end))
        (else (search-for-primes limit (if (even? start) (+ 1 start) (+ 2 start)) end))))

In [9]:
(search-for-primes 3 1000 1100)
(search-for-primes 3 10000 10100)
(search-for-primes 3 100000 100100)
(search-for-primes 3 1000000 1000100)

1009 *** 0.0009765625
1013 *** 0.0
1019 *** 0.0009765625
10007 *** 0.0009765625
10009 *** 0.001953125
10037 *** 0.001220703125
100003 *** 0.00390625
100019 *** 0.004150390625
100043 *** 0.00390625
1000003 *** 0.011962890625
1000033 *** 0.010986328125
1000037 *** 0.011962890625


#t

$\sqrt{10}$ is 3.16227766
| range| average | times $\sqrt{10}$ |
|---|---|---|
| 1000| 0.000976 | 0.003086 |
| 10000 | 0.00195 | 0.006166|
| 100000| 0.00415| 0.013123|
| 1000000| 0.01196| ---|

While the output is faster (and probably machine dependent) the `x 10`s growth runtime is proportional to $\sqrt{10}$ times across the range.

### Exercise 1.23: 
The `smallest-divisor` procedure shown at the start of this section does lots of needless testing: After it checks to see if the number is divisible by 2 there is no point in checking to see if it is divisible by any larger even numbers. This suggests that the values used for `test-divisor` should not be 2, 3, 4, 5, 6, . . ., but rather 2, 3, 5, 7, 9, . . .. To implement this change, define a procedure next that returns 3 if its input is equal to 2 and otherwise returns its input plus 2. Modify the `smallest-divisor` procedure to use `(next test-divisor)` instead of `(+ test-divisor 1)`. With timed-prime-test incorporating this modified version of `smallest-divisor`, run the test for each of the 12 primes found in Exercise 1.22. Since this modification halves the number of test steps, you should expect it to run about twice as fast. Is this expectation confirmed? If not, what is the observed ratio of the speeds of the two algorithms, and how do you explain the fact that it is different from 2?

#### Answer:

In [10]:
(define (next n) (if (even? n) (+ 1 n) (+ 2 n)))
(define (smallest-divisor n)
  (define (find-divisor test-divisor)
    (cond ((> (square test-divisor) n) n)
          ((divides? test-divisor n) test-divisor)
          (else (find-divisor (next test-divisor)))))
  (find-divisor 2))

In [11]:
(search-for-primes 3 1000 1100)
(search-for-primes 3 10000 10100)
(search-for-primes 3 100000 100100)
(search-for-primes 3 1000000 1000100)

1009 *** 0.0009765625
1013 *** 0.0
1019 *** 0.0
10007 *** 0.0
10009 *** 0.0009765625
10037 *** 0.0009765625
100003 *** 0.001953125
100019 *** 0.002197265625
100043 *** 0.001953125
1000003 *** 0.006103515625
1000033 *** 0.005859375
1000037 *** 0.007080078125


#t

| range| average | average (optimized procedure) |
|---|---|---|
| 1000| 0.000976 | 0.000976 |
| 10000 | 0.00195 | 0.000976|
| 100000| 0.00415| 0.0029296|
| 1000000| 0.01196| 0.007080|

The execution time tends towards a reduction in half as it processes larger numbers.

### Exercise 1.24: 
Modify the timed-prime-test procedure of Exercise 1.22 to use `fast-prime`? (the Fermat method), and test each of the 12 primes you found in that exercise. Since the Fermat test has `Θ(log n)` growth, how would you expect the time to test primes near 1,000,000 to compare with the time needed to test primes near 1000? Do your data bear this out? Can you explain any discrepancy you find?

#### Answer:

In [12]:
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp) (remainder (square (expmod base (/ exp 2) m)) m))
        (else (remainder (* base (expmod base (- exp 1) m)) m))))

(define (fermat-test n)
  (define (try-it a)
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1)))))

(define (fast-prime? n times)
  (cond ((= times 0) #t)
        ((fermat-test n) (fast-prime? n (- times 1)))
        (else #f)))

(define (start-prime-test n start-time)
  (if (fast-prime? n 3)
      (report-prime n (- (runtime) start-time))
      #f))

(search-for-primes 3 1000 1100)
(search-for-primes 3 10000 10100)
(search-for-primes 3 100000 100100)
(search-for-primes 3 1000000 1000100)

1009 *** 0.0009765625
1013 *** 0.0009765625
1019 *** 0.0009765625
10007 *** 0.002197265625
10009 *** 0.001953125
10037 *** 0.0009765625
100003 *** 0.001953125
100019 *** 0.0009765625
100043 *** 0.001953125
1000003 *** 0.0009765625
1000033 *** 0.001953125
1000037 *** 0.0009765625


#t

The growth of `fast-prime?` procedure is logarithmic and $\log 10$'s value is 1. i.e. we would expect a constant execution time irrespective of the number.

This observation is proven from the numbers seen:
| range| average |
|---|---|
| 1000| 0.000976 |
| 10000 | 0.000976 |
| 100000| 0.000976|
| 1000000| 0.000976|

### Exercise 1.25: 
Alyssa P. Hacker complains that we went to a lot of extra work in writing `expmod`. After all, she says, since we already know how to compute exponentials, we could have simply written
```scheme
(define (expmod base exp m)
  (remainder (fast-expt base exp) m))
```
Is she correct? Would this procedure serve as well for our fast prime tester? Explain.

#### Answer:
She is not correct. In the `expmod` function we compute the remainder of $b^{\frac{e}{2}}$ modulo `m`, square this, and take the remainder modulo `m`. This technique is useful because it means we can perform our computation without ever having to deal with numbers much larger than `m`. In Alyssa P. Hacker's version, we take the modulo *after* computing the exponential number. This will be compute intensive - for the same about of steps.

### Exercise 1.26: 
Louis Reasoner is having great difficulty doing Exercise 1.24. His `fast-prime?` test seems to run more slowly than his `prime?` test. Louis calls his friend Eva Lu Ator over to help. When they examine Louis’s code, they find that he has rewritten the expmod procedure to use an explicit multiplication, rather than calling square:
```scheme
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp)
         (remainder (* (expmod base (/ exp 2) m)
                       (expmod base (/ exp 2) m))
                    m))
        (else
         (remainder (* base
                        (expmod base (- exp 1) m))
                    m))))
```
“I don’t see what difference that could make,” says Louis. “I do.” says Eva. “By writing the procedure like that, you have transformed the `Θ(log n)` process into a `Θ(n)` process.” Explain.

#### Answer:
- Since scheme follows *applicative-order* execution, the explicit multiplication procedure would invoke `expmod` twice for every invocation, resulting in $2^n$ invocations.
- The order of growth becomes $\theta(\log(2^n))$, which is $\theta(n \log 2)$, which is $\theta(n)$

### Exercise 1.27: 
Demonstrate that the Carmichael numbers listed in Footnote 1.47 really do fool the Fermat test. that is, write a procedure that takes an integer *n* and tests whether $a^n$ is congruent to a modulo *n* for every $a < n$, and try your procedure on the given Carmichael numbers.

#### Answer:
Modifying `fermat-test` to suit the problem.

In [13]:
(define (carmichael-number? n)
  (define (try-it acc)
    (cond ((= acc 1) #t)
          ((not (= (expmod acc n n) acc)) #f)
          (else (try-it (- acc 1)))))
  (try-it (- n 1)))

(display (carmichael-number? 561))(newline)
(display (carmichael-number? 1105))(newline)
(display (carmichael-number? 1729))(newline)
(display (carmichael-number? 2465))(newline)
(display (carmichael-number? 2821))(newline)
(display (carmichael-number? 6601))(newline)

#t
#t
#t
#t
#t
#t


### Exercise 1.28: 
One variant of the Fermat test that cannot be fooled is called the Miller-Rabin test (Miller 1976; Rabin 1980). This starts from an alternate form of Fermat’s Little Theorem, which states that if $n$ is a prime number and $a$ is any positive integer less than $n$, then $a$ raised to the $(n-1)^{st}$ power is congruent to 1 modulo n. To test the primality of a number n by the Miller-Rabin test, we pick a random number $a < n$ and raise a to the $(n-1)^{st}$ power modulo n using the expmod procedure. However, whenever we perform the squaring step in `expmod`, we check to see if we have discovered a “nontrivial square root of 1 modulo n,” that is, a number not equal to 1 or n-1 whose square is equal to 1 modulo n. It is possible to prove that if such a nontrivial square root of 1 exists, then $n$ is not prime. It is also possible to prove that if n is an odd number that is not prime, then, for at least half the numbers $a < n$, computing $a^{n-1}$ in this way will reveal a nontrivial square root of 1 modulo n. (This is why the Miller-Rabin test cannot be fooled.) Modify the `expmod` procedure to signal if it discovers a nontrivial square root of 1, and use this to implement the Miller-Rabin test with a procedure analogous to fermat-test. Check your procedure by testing various known primes and non-primes. Hint: One convenient way to make expmod signal is to have it return 0

#### Answer
This is fairly straightforward if you were able to follow along the hints given in the problem.

In [18]:
(define (remainder-mr-test x y)
  (if (and (not (= x 1))
           (not (= x (- y 1)))
           (= (remainder (* x x) y) 1))
      0
      (remainder (* x x) y)))

(define (expmod-mr-test base exp m)
  (cond ((= exp 0) 1)
        ((even? exp) (remainder-mr-test (expmod-mr-test base (/ exp 2) m) m))
        (else (remainder (* base (expmod-mr-test base (- exp 1) m)) m))))

(define (miller-rabin-test n)
  (define (try-it a)
    (= (expmod-mr-test a (- n 1) n) 1))
  (try-it (+ 1 (random (- n 1)))))

(define (fast-prime-mr-test? n times)
  (cond ((= times 0) #t)
        ((miller-rabin-test n) (fast-prime-mr-test? n (- times 1)))
        (else #f)))

(display (fast-prime? 561 1)) (newline)
(display (fast-prime-mr-test? 561 1))

#t
#f